In [1]:
print('get qualified txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import flipside_utils as f
import clickhouse_utils as ch
import csv_utils as cu
import google_bq_utils as bqu
sys.path.pop()

import time

import numpy as np
import pandas as pd
import os
import clickhouse_connect as cc

get qualified txs


In [2]:
ch_client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

query_name = 'daily_evms_qualified_txs_counts'

In [3]:
col_list = [
        'dt','blockchain','name','layer','chain_id'
        , 'num_raw_txs', 'num_success_txs','num_qualified_txs','source'
        ]

In [16]:
trailing_days = 90
flipside_configs = [
        {'blockchain': 'blast', 'chain_id': 238, 'name': 'Blast', 'layer': 'L2'}
]
clickhouse_configs = [
        {'blockchain': 'metal', 'chain_id': 1750,'name': 'Metal', 'layer': 'L2'},
        {'blockchain': 'mode', 'chain_id': 34443,'name': 'Mode', 'layer': 'L2'},
        {'blockchain': 'bob', 'chain_id': 60808,'name': 'BOB (Build on Bitcoin)', 'layer': 'L2'},
        {'blockchain': 'fraxtal', 'chain_id': 252,'name': 'Fraxtal', 'layer': 'L2'},
        {'blockchain': 'cyber', 'chain_id': 7560,'name': 'Cyber', 'layer': 'L2'},
        {'blockchain': 'mint', 'chain_id': 185,'name': 'Mint', 'layer': 'L2'},
]

In [5]:
print('     flipside runs')
flip_dfs = []
with open(os.path.join("inputs/sql/flipside_bychain.sql"), "r") as file:
    og_query = file.read()

for chain in flipside_configs:
    print('     flipside: ' + chain['blockchain'])
    query = og_query
    # Pass in Params to the query
    query = query.replace("@blockchain@", chain['blockchain'])
    query = query.replace("@chain_id@", str(chain['chain_id']))
    query = query.replace("@name@", chain['name'])
    query = query.replace("@layer@", chain['layer'])
    query = query.replace("@trailing_days@", str(trailing_days))
    
    try:
        df = f.query_to_df(query)
        flip_dfs.append(df)
    except Exception as e:  # Use FlipsideError if available instead of Exception
        print(f"Error querying Flipside for {chain['blockchain']}: {str(e)}")
        print("Skipping this chain due to API credit limitation or other issues.")
        continue

if flip_dfs:
    flip = pd.concat(flip_dfs)
    flip['source'] = 'flipside'
    flip['dt'] = pd.to_datetime(flip['dt']).dt.tz_localize(None)
    flip = flip[col_list]
else:
    print("No data was retrieved from Flipside. The resulting DataFrame will be empty.")
    flip = pd.DataFrame(columns=col_list)

     flipside runs
     flipside: blast


In [6]:
# Run Dune
print('     dune runs')
days_param = d.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')
dune_df = d.get_dune_data(query_id = 3740822, #https://dune.com/queries/3740822
    name = "dune_evms_qualified_txs",
    path = "outputs",
    performance="large",
    params = [days_param],
    num_hours_to_rerun=12
)
dune_df['source'] = 'dune'
dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)
dune_df = dune_df[col_list]

2024-07-09 18:09:27.651 | INFO     | duneapi_utils:get_dune_data:58 - Results available at https://dune.com/queries/3740822?trailing_days=90


     dune runs


2024-07-09 18:09:28,834 INFO dune_client.api.base executing 3740822 on large cluster
2024-07-09 18:09:29,188 INFO dune_client.api.base waiting for query execution 01J2CRPTD1E3CJ115AKFSDXVV2 to complete: ExecutionState.PENDING (queue position: 1)
2024-07-09 18:09:30,371 INFO dune_client.api.base waiting for query execution 01J2CRPTD1E3CJ115AKFSDXVV2 to complete: ExecutionState.PENDING (queue position: 1)
2024-07-09 18:09:31,546 INFO dune_client.api.base waiting for query execution 01J2CRPTD1E3CJ115AKFSDXVV2 to complete: ExecutionState.PENDING (queue position: 1)
2024-07-09 18:09:32,710 INFO dune_client.api.base waiting for query execution 01J2CRPTD1E3CJ115AKFSDXVV2 to complete: ExecutionState.PENDING (queue position: 1)
2024-07-09 18:09:33,882 INFO dune_client.api.base waiting for query execution 01J2CRPTD1E3CJ115AKFSDXVV2 to complete: ExecutionState.PENDING (queue position: 1)
2024-07-09 18:09:35,048 INFO dune_client.api.base waiting for query execution 01J2CRPTD1E3CJ115AKFSDXVV2 to co

In [17]:
# Run Clickhouse
print('     clickhouse runs')
ch_dfs = []
with open(os.path.join("inputs/sql/goldsky_bychain.sql"), "r") as file:
                        og_query = file.read()

for chain in clickhouse_configs:
        print(     'clickhouse: ' + chain['blockchain'])
        query = og_query
        #Pass in Params to the query
        query = query.replace("@blockchain@", chain['blockchain'])
        if chain['blockchain'] == 'bob':
                query = query.replace("chain_id, --db chain_id", str(chain['chain_id']) + " as chain_id,")
                # query = query.replace("@chain_id@", str(chain['chain_id']))
        query = query.replace("@name@", chain['name'])
        query = query.replace("@layer@", chain['layer'])
        query = query.replace("@trailing_days@", str(trailing_days))
        
        df = ch_client.query_df(query)

        ch_dfs.append(df)

ch = pd.concat(ch_dfs)
ch['source'] = 'goldsky'
ch['dt'] = pd.to_datetime(ch['dt']).dt.tz_localize(None)
ch = ch[col_list]

     clickhouse runs
clickhouse: metal
clickhouse: mode
clickhouse: bob


DatabaseError: :HTTPDriver for https://pdmv9lhojy.us-west-2.aws.clickhouse.cloud:8443 returned response code 400)
 Code: 62. DB::Exception: Syntax error: failed at position 1562 ('as') (line 43, col 22): as chain_id = l.'60808' as chain_id
        AND r.dt = l.dt

 FORMAT Native. Expected one of: OR, AND, IS NOT DISTINCT FROM, IS NULL, IS NOT NULL, BETWEEN, NOT BETWEEN, LIKE, ILIKE, NOT LIKE, NOT ILIKE, REGEXP, IN, NOT IN, GLOBAL IN, GLOBAL NOT IN, MOD, DIV, table, table function, subquery or list of joined tables, array join, LEFT ARRAY JOIN, INNER, ARRAY JOIN, JOIN, PREWHERE, WHERE, GROUP BY, WITH, HAVING, WINDOW, ORDER BY, LIMIT, OFFSET, FETCH, SETTINGS, UNION, EXCEPT, INTERSECT, INTO OUTFILE, FORMAT, end of query. (SYNTAX_ERROR) (version 24.2.2.16370 (official build))


In [10]:
# Step 1: Filter dune_df for chains not in flip
filtered_dune_df = dune_df[~dune_df['chain_id'].isin(flip['chain_id'])]
# Step 2: Union flip and filtered_dune_df
combined_flip_dune = pd.concat([flip, filtered_dune_df])
# Step 3: Filter ch for chains not in combined_flip_dune
filtered_ch = ch[~ch['chain_id'].isin(combined_flip_dune['chain_id'])]
# Step 4: Union the result with filtered_ch
final_df = pd.concat([combined_flip_dune, filtered_ch])
# final_df


In [11]:
opstack_metadata = pd.read_csv('../op_chains_tracking/outputs/chain_metadata.csv')

opstack_metadata['chain_id'] = opstack_metadata['mainnet_chain_id']

meta_cols = ['is_op_chain','op_based_version', 'chain_id', 'alignment','chain_name', 'display_name']

print("Columns in opstack_metadata:", opstack_metadata.columns)
print("Columns in opstack_metadata[meta_cols]:", opstack_metadata[meta_cols].columns)
print("Columns in final_df:", final_df.columns)


Columns in opstack_metadata: Index(['chain_name', 'display_name', 'mainnet_chain_id',
       'public_mainnet_launch_date', 'op_based_version', 'chain_type',
       'has_mods', 'is_law_of_chains', 'raas_deployer', 'rpc_url',
       'product_website', 'defillama_slug', 'l2beat_slug',
       'growthepie_origin_key', 'goldsky_schema', 'dune_schema', 'gas_token',
       'chain_layer', 'da_layer', 'output_root_layer', 'system_config_proxy',
       'is_in_superchain_registry_repo', 'oplabs_db_schema', 'batchinbox_from',
       'batchinbox_to', 'outputoracle_from', 'outputoracle_to_proxy',
       'l1_standard_bridge', 'dispute_game_factory', 'is_op_chain',
       'alignment', 'chain_id'],
      dtype='object')
Columns in opstack_metadata[meta_cols]: Index(['is_op_chain', 'op_based_version', 'chain_id', 'alignment',
       'chain_name', 'display_name'],
      dtype='object')
Columns in final_df: Index(['dt', 'blockchain', 'name', 'layer', 'chain_id', 'num_raw_txs',
       'num_success_txs', 'nu

In [12]:
final_enriched_df = final_df.merge(opstack_metadata[meta_cols], on='chain_id', how = 'left')
final_enriched_df['alignment'] = final_enriched_df['alignment'].fillna('Other EVMs')
final_enriched_df['is_op_chain'] = final_enriched_df['is_op_chain'].fillna(False)
final_enriched_df['display_name'] = final_enriched_df['display_name'].fillna(final_enriched_df['name'])

final_enriched_df = final_enriched_df.drop(columns=['name'])

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_22246/1364696242.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_enriched_df['is_op_chain'] = final_enriched_df['is_op_chain'].fillna(False)


In [13]:
final_enriched_df.sort_values(by=['dt','blockchain'], ascending =[False, False], inplace = True)

final_enriched_df.to_csv('outputs/'+query_name+'.csv', index=False)

In [14]:
final_enriched_df['chain_id'] = final_enriched_df['chain_id'].astype(int)
final_enriched_df['num_raw_txs'] = final_enriched_df['num_raw_txs'].astype(int)
final_enriched_df['num_success_txs'] = final_enriched_df['num_success_txs'].astype(int)
final_enriched_df['num_qualified_txs'] = final_enriched_df['num_qualified_txs'].astype(int)
# final_enriched_df.dtypes

In [15]:
#BQ Upload
bqu.append_and_upsert_df_to_bq_table(final_enriched_df, query_name, unique_keys = ['chain_id','dt'])

Start Writing api_table_uploads.daily_evms_qualified_txs_counts_staging
Data loaded successfully to api_table_uploads.daily_evms_qualified_txs_counts_staging


2024-07-09 18:22:59,074 INFO google_bq_utils Append and upsert to api_table_uploads.daily_evms_qualified_txs_counts completed successfully
2024-07-09 18:22:59,314 INFO google_bq_utils Staging table oplabs-tools-data.api_table_uploads.daily_evms_qualified_txs_counts_staging deleted.
